In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15837808
paper_name = 'irwin_sandmeyer_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [31]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1')

In [32]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 70 x 2


In [33]:
original_data.head()

,Deletion Reduced Transposition:,Deletion Enhanced Transposition
0,FUR4,LDB7
1,PKR1,ECM29
2,FTR1,TIR3
3,FET3,HTL1
4,EAF7,ADA2


In [34]:
original_data1 = original_data[['Deletion Reduced Transposition:']].copy()
original_data1['data'] = -1
original_data1.columns=['gene','data']

original_data2 = original_data[['Deletion Enhanced Transposition']].copy()
original_data2['data'] = 1
original_data2.columns=['gene','data']

In [35]:
original_data = pd.concat([original_data1, original_data2], axis=0)

In [36]:
original_data.head()

,gene,data
0,FUR4,-1
1,PKR1,-1
2,FTR1,-1
3,FET3,-1
4,EAF7,-1


In [37]:
original_data['gene'] = original_data['gene'].astype(str)

In [38]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [40]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'].values, to='orf')

In [41]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            gene  data  orf
index_input                
64           NAN     1  NAN
65           NAN     1  NAN
66           NAN     1  NAN
67           NAN     1  NAN
68           NAN     1  NAN
69           NAN     1  NAN


In [42]:
original_data = original_data.loc[t,:]

In [43]:
original_data.set_index('orf', inplace=True)

In [44]:
original_data = original_data[['data']].copy()

In [45]:
original_data = original_data.groupby(original_data.index).mean()

In [46]:
original_data.shape

(130, 1)

# Prepare the final dataset

In [47]:
data = original_data.copy()

In [48]:
dataset_ids = [182]
datasets = datasets.reindex(index=dataset_ids)

In [49]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [50]:
data.head()

dataset_id,182
data_type,value
orf,
YAL013W,-1
YAL027W,-1
YAR003W,1
YBL006C,1
YBL007C,-1


## Subset to the genes currently in SGD

In [51]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [52]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,182
,data_type,value
gene_id,orf,
11,YAL013W,-1
25,YAL027W,-1
63,YAR003W,1
94,YBL006C,1
95,YBL007C,-1


# Normalize

In [53]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [54]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [55]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        182          
data_type       value    valuez
gene_id orf                    
11      YAL013W    -1 -5.853197
25      YAL027W    -1 -5.853197
63      YAR003W     1  5.972650
94      YBL006C     1  5.972650
95      YBL007C    -1 -5.853197

# Print out

In [56]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [57]:
from IO.save_data_to_db3 import *

In [58]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15837808...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  2.49it/s]

Updating the data_modified_on field...
